<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="../../resources/logo.png" alt="Intellinum Bootcamp" style="width: 600px; height: 163px">
</div>

# Advanced User Defined Functions

Apache Spark&trade; allows you to create your own User Defined Functions (UDFs) specific to the needs of your data.

## In this lesson you:
* Apply UDFs with a multiple DataFrame column inputs
* Apply UDFs that return complex types
* Write vectorized UDFs using Python


### Complex Transformations
 
UDFs provide custom, generalizable code that you can apply to ETL workloads when Spark's built-in functions won't suffice.  
In the last lesson we covered a simple version of this: UDFs that take a single DataFrame column input and return a primitive value. Often a more advanced solution is needed.

UDFs can take multiple column inputs. While UDFs cannot return multiple columns, they can return complex, named types that are easily accessible. This approach is especially helpful in ETL workloads that need to clean complex and challenging data structures.

Another other option is the new vectorized, or pandas, UDFs available in Spark 2.3. These allow for more performant UDFs written in Python.<br><br>

<div><img src="../../resources/pandas-udfs.png" style="height: 400px; margin: 20px"/></div>

### UDFs with Multiple Columns

To begin making more complex UDFs, start by using multiple column inputs.  This is as simple as adding extra inputs to the function or lambda you convert to the UDF.

Run the following cell to create the lab environment:

In [1]:
#MODE = "LOCAL"
MODE = "CLUSTER"

import sys
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkConf
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
from matplotlib import interactive
interactive(True)
import matplotlib.pyplot as plt
%matplotlib inline
import json
import math
import numbers
import numpy as np
import plotly
plotly.offline.init_notebook_mode(connected=True)

sys.path.insert(0,'../../src')
from settings import *

try:
    fh = open('../../libs/pyspark24_py36.zip', 'r')
except FileNotFoundError:
    !aws s3 cp s3://devops.intellinum.co/bins/pyspark24_py36.zip ../../libs/pyspark24_py36.zip

try:
    spark.stop()
    print("Stopped a SparkSession")
except Exception as e:
    print("No existing SparkSession detected")
    print("Creating a new SparkSession")

SPARK_DRIVER_MEMORY= "1G"
SPARK_DRIVER_CORE = "1"
SPARK_EXECUTOR_MEMORY= "1G"
SPARK_EXECUTOR_CORE = "1"
SPARK_EXECUTOR_INSTANCES = 12



conf = None
if MODE == "LOCAL":
    os.environ["PYSPARK_PYTHON"] = "/home/yuan/anaconda3/envs/pyspark24_py36/bin/python"
    conf = SparkConf().\
            setAppName("pyspark_etl_08-advanced-udf").\
            setMaster('local[*]').\
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars', '../../libs/mysql-connector-java-5.1.45-bin.jar').\
            set('spark.jars.packages','net.java.dev.jets3t:jets3t:0.9.0,com.google.guava:guava:16.0.1,com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1')
else:
    os.environ["PYSPARK_PYTHON"] = "./MN/pyspark24_py36/bin/python"
    conf = SparkConf().\
            setAppName("pyspark_etl_08-advanced-udf").\
            setMaster('yarn-client').\
            set('spark.executor.cores', SPARK_EXECUTOR_CORE).\
            set('spark.executor.memory', SPARK_EXECUTOR_MEMORY).\
            set('spark.driver.cores', SPARK_DRIVER_CORE).\
            set('spark.driver.memory', SPARK_DRIVER_MEMORY).\
            set("spark.executor.instances", SPARK_EXECUTOR_INSTANCES).\
            set('spark.sql.files.ignoreCorruptFiles', 'true').\
            set('spark.yarn.dist.archives', '../../libs/pyspark24_py36.zip#MN').\
            set('spark.sql.shuffle.partitions', '5000').\
            set('spark.default.parallelism', '5000').\
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars.packages','net.java.dev.jets3t:jets3t:0.9.0,com.google.guava:guava:16.0.1,com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.1'). \
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars', 's3://devops.intellinum.co/bins/mysql-connector-java-5.1.45-bin.jar')
        

spark = SparkSession.builder.\
    config(conf=conf).\
    getOrCreate()


sc = spark.sparkContext

sc.addPyFile('../../src/settings.py')

sc=spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version", "2")

def display(df, limit=10):
    return df.limit(limit).toPandas()

def dfTest(id, expected, result):
    assert str(expected) == str(result), "{} does not equal expected {}".format(result, expected)

No existing SparkSession detected
Creating a new SparkSession


Write a basic function that combines two columns.

In [2]:
def manual_add(x, y):
    return x + y

manual_add(1, 2)

3

Register the function as a UDF by binding it with a Python variable, adding a name to access it in the SQL API and giving it a return type.

In [4]:
from pyspark.sql.types import IntegerType

manualAddPythonUDF = spark.udf.register("manualAddSQLUDF", manual_add, IntegerType())

Create a dummy DataFrame to apply the UDF.

In [5]:
integerDF = (spark.createDataFrame([
  (1, 2),
  (3, 4),
  (5, 6)
], ["col1", "col2"]))

display(integerDF)

,col1,col2
0,1,2
1,3,4
2,5,6


Apply the UDF to your DataFrame.

In [6]:
integerAddDF = integerDF.select("*", manualAddPythonUDF("col1", "col2").alias("sum"))

display(integerAddDF)

,col1,col2,sum
0,1,2,3
1,3,4,7
2,5,6,11


### UDFs with Complex Output

Complex outputs are helpful when you need to return multiple values from your UDF. The UDF design pattern involves returning a single column to drill down into, to pull out the desired data.

Start by determining the desired output.  This will look like a schema with a high level `StructType` with numerous `StructFields`.

For a refresher on this, see the lesson **Applying Schemas to JSON Data** in <a href="./02-applying-schemas-to-JSON.ipynb" target="_blank">ETL Part 1 course</a>.

In [7]:
from pyspark.sql.types import FloatType, StructType, StructField

mathOperationsSchema = StructType([
  StructField("sum", FloatType(), True), 
  StructField("multiplication", FloatType(), True), 
  StructField("division", FloatType(), True) 
])

Create a function that returns a tuple of your desired output.

In [8]:
def manual_math(x, y):
    return (float(x + y), float(x * y), x / float(y))

manual_math(1, 2)

(3.0, 2.0, 0.5)

Register your function as a UDF and apply it.  In this case, your return type is the schema you created.

In [9]:
manualMathPythonUDF = spark.udf.register("manualMathSQLUDF", manual_math, mathOperationsSchema)

display(integerDF.select("*", manualMathPythonUDF("col1", "col2").alias("sum")))

,col1,col2,sum
0,1,2,"(3.0, 2.0, 0.5)"
1,3,4,"(7.0, 12.0, 0.75)"
2,5,6,"(11.0, 30.0, 0.8333333134651184)"


### Vectorized UDFs in Python

Starting in Spark 2.3, vectorized UDFs can be written in Python called Pandas UDFs.  This alleviates some of the serialization and invocation overhead of conventional Python UDFs.  While there are a number of types of these UDFs, this walk-through focuses on scalar UDFs. This is an ideal solution for Data Scientists needing performant UDFs written in Python.

:NOTE: Your cluster will need to run Spark 2.3 in order to execute the following code.

Use the decorator syntax to designate a Pandas UDF.  The input and outputs are both Pandas series of doubles.

In [10]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf('double', PandasUDFType.SCALAR)
def pandas_plus_one(v):
    return v + 1

Create a DataFrame to apply the UDF.

In [11]:
from pyspark.sql.functions import col, rand

df = spark.range(0, 10 * 1000 * 1000)

display(df)

,id
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


Apply the UDF

In [12]:
display(df.withColumn('id_transformed', pandas_plus_one("id")))

,id,id_transformed
0,0,1.0
1,1,2.0
2,2,3.0
3,3,4.0
4,4,5.0
5,5,6.0
6,6,7.0
7,7,8.0
8,8,9.0
9,9,10.0


## Exercise 1: Multiple Column Inputs to Complex Type

Given a DataFrame of weather in various units, write a UDF that translates a column for temperature and a column for units into a complex type for temperature in three units:<br><br>

* fahrenheit
* celsius
* kelvin

### Step 1: Import and Explore the Data

Import the data sitting in `s3a://data.intellinum.co/bootcamp/common/weather/StationData/stationData.parquet` and save it to `weatherDF`.

In [13]:
# TODO

In [15]:
# TEST - Run this cell to test your solution
cols = set(weatherDF.columns)

dfTest("ET2-P-04-01-01", 2559, weatherDF.count())
dfTest("ET2-P-04-01-02", True, "TAVG" in cols and "UNIT" in cols)

print("Tests passed!")

Tests passed!


### Step 2: Define Complex Output Type

Define the complex output type for your UDF.  This should look like the following:

| Field Name | Type |
|:-----------|:-----|
| fahrenheit | Float |
| celsius | Float |
| kelvin | Float |

In [16]:
# TODO

In [18]:
# TEST - Run this cell to test your solution
from pyspark.sql.types import FloatType
names = [i.name for i in schema.fields]

dfTest("ET2-P-04-02-01", 3, len(schema.fields))
dfTest("ET2-P-04-02-02", [FloatType(), FloatType(), FloatType()], [i.dataType for i in schema.fields])
dfTest("ET2-P-04-02-03", True, "fahrenheit" in names and "celsius" in names and "kelvin" in names)

print("Tests passed!")

Tests passed!


### Step 3: Create the Function

Create a function that takes `temperature` as a Float and `unit` as a String.  `unit` will either be `F` for fahrenheit or `C` for celsius.  
Return a tuple of floats of that value as `(fahrenheit, celsius, kelvin)`.

Use the following equations:

| From | To Fahrenheit | To Celsius | To Kelvin |
|:-----|:--------------|:-----------|:-----------|
| Fahrenheit | F | (F - 32) * 5/9 | (F - 32) * 5/9 + 273.15 |
| Celsius | (C * 9/5) + 32 | C | C + 273.15 |
| Kelvin | (K - 273.15) * 9/5 + 32 | K - 273.15 | K |

In [19]:
# TODO


In [21]:
# TEST - Run this cell to test your solution
dfTest("ET2-P-04-03-01", (194.0, 90, 363.15), temperatureConverter(90, "C"))
dfTest("ET2-P-04-03-02", (0, -17.77777777777778, 255.3722222222222), temperatureConverter(0, "F"))

print("Tests passed!")

Tests passed!


### Step 4: Register the UDF

Register the UDF as `temperatureConverterUDF`

In [ ]:
# TODO

In [24]:
# TEST - Run this cell to test your solution
dfTest("ET2-P-04-04-01", (194.0, 90, 363.15), temperatureConverterUDF.func(90, "C"))
dfTest("ET2-P-04-04-02", (0, -17.77777777777778, 255.3722222222222), temperatureConverterUDF.func(0, "F"))

print("Tests passed!")

Tests passed!


### Step 5: Apply your UDF

Create `weatherEnhancedDF` with a new column `TAVGAdjusted` that applies your UDF.

In [ ]:
# TODO

In [26]:
# TEST - Run this cell to test your solution
result = weatherEnhancedDF.select("TAVGAdjusted").first()[0].asDict()

dfTest("ET2-P-04-05-01", {'fahrenheit': 61.0, 'celsius': 16.11111068725586, 'kelvin': 289.2611083984375}, result)
dfTest("ET2-P-04-05-02", 2559, weatherEnhancedDF.count())

print("Tests passed!")

Tests passed!


## Review

**Question:** How do UDFs handle multiple column inputs and complex outputs?   
**Answer:** UDFs allow for multiple column inputs.  Complex outputs can be designated with the use of a defined schema encapsulate in a `StructType()` or a Scala case class.

**Question:** How can I do vectorized UDFs in Python and are they as performant as built-in functions?   
**Answer:** Spark 2.3 includes the use of vectorized UDFs using Pandas syntax. Even though they are vectorized, these UDFs will not be as performant built-in functions, though they will be more performant than non-vectorized Python UDFs.

&copy; 2019 [Intellinum Analytics, Inc](http://www.intellinum.co). All rights reserved.<br/>